# The Best Possible Location to Open a New Restaurant in Asheville

## Introduction

Asheville is a city located in North Carolina, it has a population of approximately 92,000 people according to estimates defined by the US Census Bureau as of July 1, 2018. I choose this city for this project based on information checked on the city council page, the demographics of the city, and the city geographic location.

The city council of Asheville established in January 2016 a 20-year vision plan for the city, for me that is important because it is a sign of commitment towards future investments made in the city. Some of the topics included in the city plan are: being an equitable and diverse community, being a clean and healthy environment, being a connected and engaged community among other topics. However, the one that I find more important regarding business activity, like a potential new restaurant, is the commitment made by the city council to be a financially resilient city.

The demographics of the city defined by estimates of the US Census Bureau described a diverse city. As of July 1, 2018, the demographics include Asian, Black/African American, Native Hawaiian and Other Pacific Islander, White, and Hispanic/Latino. This diversity in races and the relatively small total population number show a potential location for a new restaurant opening. It is worth considering that the total population will increase once the 2020 census concludes.

Finally the geographic location of Asheville makes it a potential location for new restaurants, it has one of the fastest growing regional airports. According to the airport 2019 annual report the growth is representative of the vitality of western North Carolina and the commitment of the region’s travelers to “fly local.” This is another indicator of the potential growth in tourism for the city, and with that, another good reason to open a new restaurant.

# Data Section

The data used to find the best possible location to open a new restaurant in Asheville, North Carolina will be extracted from the City of Asheville GIS web page, it is classified by neighborhood and it has a GeoJSON file to download and get the official coordinates for each neighborhood. My analysis will describe the current venues in Asheville's neighborhoods obtained through the Foursquare API. 

Using Foursquare API I will get information regarding the venues in Asheville's neighborhoods, with this I will recommend the best possible neighborhood to open a new restaurant. My recommendation will be based on the potential costumers the neighborhood would have for this new restaurant, the potential competitors, and the diversity of the chosen neighborhood.

The final goal of the project will not only be to establish the best possible neighborhood in Asheville but also to give a recommendation of the type of cuisine for this new restaurant. The recommendation will be based on the most common restaurants found in Asheville's neighborhoods, to establish the most popular and the less popular ones.